# GRACE as Encoder

In [63]:
import time
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from model_contrastive import Encoder, GRACE, drop_feature
from model import CNLinkPredictor, GCN
from NeighborOverlap import train, test
from ogbdataset import loaddataset, randomsplit
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
from torch.utils.tensorboard import SummaryWriter
from utils_contrastive import compute_pr, eigenvector_centrality
from torch_geometric.utils import dropout_adj, to_undirected, degree
from functional_GCA import drop_edge_weighted, pr_drop_weights, degree_drop_weights, evc_drop_weights, compute_pr, eigenvector_centrality, feature_drop_weights, drop_feature_weighted_2

In [55]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [56]:
hp = {
    'xdp': 0.7,
    'tdp': 0.3,
    'pt': 0.75,
    'gnnedp': 0.0,
    'preedp': 0.4,
    'predp': 0.05,
    'gnndp': 0.05,
    'probscale': 4.3,
    'proboffset': 2.8,
    'alpha': 1.0,
    'gnnlr': 0.0043,
    'prelr': 0.0024,
    'batch_size': 1152,
    'ln': True,
    'lnnn': True,
    'epochs': 100,
    'runs': 1,
    'hiddim': 256,
    'mplayers': 1,
    'testbs': 8192,
    'maskinput': True,
    'jk': True,
    'use_xlin': True,
    'tailact': True,
}
device = torch.device(f'cuda' if torch.cuda.is_available() else 'cpu')

## legacy encoder

In [7]:
def run_legacy(r, dataset, evaluator, hp):
    writer = SummaryWriter(f"./rec/GCN_NCN")
    writer.add_text("hyperparams", str(hp)) 
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        data, split_edge = loaddataset(dataset, False) # get a new split of dataset
        data = data.to(device)
    bestscore = None
    # build model
    model = GCN(data.num_features, hp['hiddim'], hp['hiddim'], hp['mplayers'],
                 hp['gnndp'], hp['ln'], False, data.max_x,
                 'puregcn', hp['jk'], hp['gnnedp'],  xdropout=hp['xdp'], taildropout=hp['tdp']).to(device)
    predictor = CNLinkPredictor(hp['hiddim'], hp['hiddim'], 1, 3,
                       hp['predp'], hp['preedp'], hp['lnnn']).to(device)
    optimizer = torch.optim.Adam([{'params': model.parameters(), "lr": hp['gnnlr']}, 
       {'params': predictor.parameters(), 'lr': hp['prelr']}])
    for epoch in range(1, 1 + hp['epochs']):
        legacy_train(epoch, model, predictor, data, split_edge, optimizer, evaluator, hp)
        if epoch % 100 == 0:
            legacy_test(r, epoch, model, predictor, data, split_edge, evaluator, bestscore, writer, hp)

In [8]:
def legacy_train(epoch, model, predictor, data, split_edge, optimizer, evaluator, hp):
            t1 = time.time()
            loss = train(model, predictor, data, split_edge, optimizer,
                         hp['batch_size'], hp['maskinput'], [], None)
            if epoch % 10 == 0:
                print(f"10 train time {time.time()-t1:.2f} s, loss {loss:.4f}", flush=True)

In [9]:
def legacy_test(run, epoch, model, predictor, data, split_edge, evaluator, bestscore, writer, hp):
    t1 = time.time()
    results, h = test(model, predictor, data, split_edge, evaluator,
                   8192, False)
    print(f"test time {time.time()-t1:.2f} s")
    if bestscore is None:
        bestscore = {key: list(results[key]) for key in results}
    for key, result in results.items():
        writer.add_scalars(f"{key}_{run}", {
            "trn": result[0],
            "val": result[1],
            "tst": result[2]
        }, epoch)
        train_hits, valid_hits, test_hits = result
        if valid_hits > bestscore[key][1]:
            bestscore[key] = list(result)
        print(key)
        print(f'Run: {run + 1:02d}, '
              f'Epoch: {epoch:02d}, '
              f'Train: {100 * train_hits:.2f}%, '
              f'Valid: {100 * valid_hits:.2f}%, '
              f'Test: {100 * test_hits:.2f}%')
    print('---', flush=True)

In [7]:
for dataset in  ["Cora", "Citeseer", "Pubmed"]:
    print(f'################### {dataset} #################')
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        evaluator = Evaluator(name=f'ogbl-ppa')
    else:
        evaluator = Evaluator(name=f'ogbl-{args.dataset}')
    for r in range(hp['runs']):
        set_seed(r)
        run_legacy(r, dataset, evaluator, hp)

################### Cora #################


/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


2708 tensor(2707)
dataset split 
train edge 3696
valid edge 527
valid edge_neg 1055
test edge 1055
test edge_neg 1055
10 train time 0.05 s, loss 1.1758
10 train time 0.03 s, loss 0.8270
10 train time 0.03 s, loss 0.7310
10 train time 0.03 s, loss 0.7105
10 train time 0.03 s, loss 0.6186
10 train time 0.03 s, loss 0.6194
10 train time 0.03 s, loss 0.6015
10 train time 0.03 s, loss 0.5966
10 train time 0.03 s, loss 0.5795
10 train time 0.03 s, loss 0.5656
test time 0.01 s
Hits@20
Run: 01, Epoch: 100, Train: 99.81%, Valid: 72.30%, Test: 66.35%
Hits@50
Run: 01, Epoch: 100, Train: 99.95%, Valid: 84.06%, Test: 80.85%
Hits@100
Run: 01, Epoch: 100, Train: 100.00%, Valid: 91.08%, Test: 88.25%
---
################### Citeseer #################
3327 tensor(3325)
dataset split 
train edge 3187
valid edge 455
valid edge_neg 910
test edge 910
test edge_neg 910


/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


10 train time 0.02 s, loss 1.1616
10 train time 0.02 s, loss 0.6514
10 train time 0.03 s, loss 0.5095
10 train time 0.03 s, loss 0.4186
10 train time 0.03 s, loss 0.4032
10 train time 0.02 s, loss 0.3737
10 train time 0.03 s, loss 0.3910
10 train time 0.02 s, loss 0.3347
10 train time 0.05 s, loss 0.3351
10 train time 0.04 s, loss 0.2925
test time 0.02 s
Hits@20
Run: 01, Epoch: 100, Train: 100.00%, Valid: 79.12%, Test: 77.58%
Hits@50
Run: 01, Epoch: 100, Train: 100.00%, Valid: 85.71%, Test: 85.49%
Hits@100
Run: 01, Epoch: 100, Train: 100.00%, Valid: 92.09%, Test: 91.10%
---
################### Pubmed #################


/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


19717 tensor(19716)
dataset split 
train edge 31028
valid edge 4432
valid edge_neg 8864
test edge 8864
test edge_neg 8864
10 train time 0.64 s, loss 0.5941
10 train time 0.62 s, loss 0.5149
10 train time 0.69 s, loss 0.4916
10 train time 0.61 s, loss 0.4602
10 train time 0.64 s, loss 0.4486
10 train time 0.58 s, loss 0.4398
10 train time 0.51 s, loss 0.4409
10 train time 0.66 s, loss 0.4347
10 train time 0.55 s, loss 0.4298
10 train time 0.71 s, loss 0.4264
test time 0.17 s
Hits@20
Run: 01, Epoch: 100, Train: 47.86%, Valid: 43.91%, Test: 36.92%
Hits@50
Run: 01, Epoch: 100, Train: 70.48%, Valid: 59.84%, Test: 59.06%
Hits@100
Run: 01, Epoch: 100, Train: 85.89%, Valid: 72.50%, Test: 75.55%
---


In [2]:
from ogb.linkproppred import Evaluator

evaluator = Evaluator(name = 'ogbl-ppa')
print(evaluator.expected_input_format) 
print(evaluator.expected_output_format) 

==== Expected input format of Evaluator for ogbl-ppa
{'y_pred_pos': y_pred_pos, 'y_pred_neg': y_pred_neg}
- y_pred_pos: numpy ndarray or torch tensor of shape (num_edges, ). Torch tensor on GPU is recommended for efficiency.
- y_pred_neg: numpy ndarray or torch tensor of shape (num_edges, ). Torch tensor on GPU is recommended for efficiency.
y_pred_pos is the predicted scores for positive edges.
y_pred_neg is the predicted scores for negative edges.
Note: As the evaluation metric is ranking-based, the predicted scores need to be different for different edges.
==== Expected output format of Evaluator for ogbl-ppa
{hits@100': hits@100}
- hits@100 (float): Hits@100 score



## GRACE encoder

In [14]:
def run_grace(r, dataset, evaluator, hp):
    writer = SummaryWriter(f"./rec/GRACE_NCN")
    writer.add_text("hyperparams", str(hp)) 
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        data, split_edge = loaddataset(dataset, False) # get a new split of dataset
        data = data.to(device)
    bestscore = None
    # build model
    basic_encoder = Encoder(data.num_features, hp['hiddim'], nn.Identity()).to(device)
    model = GRACE(basic_encoder, hp['hiddim'], 32).to(device)
    pretrain_grace(model, data)
    predictor = CNLinkPredictor(hp['hiddim'], hp['hiddim'], 1, 3,
                       hp['predp'], hp['preedp'], hp['lnnn']).to(device)
    optimizer = torch.optim.Adam([{'params': model.parameters(), "lr": hp['gnnlr']}, 
       {'params': predictor.parameters(), 'lr': hp['prelr']}])

    for epoch in range(1, 1 + hp['epochs']):
        legacy_train(epoch, model, predictor, data, split_edge, optimizer, evaluator, hp)
        if epoch % 100 == 0:
            legacy_test(r, epoch, model, predictor, data, split_edge, evaluator, bestscore, writer, hp)

In [15]:
def pretrain_grace(model, data):
    param = {
        'learning_rate': 0.01,
        'num_hidden': 256,
        'num_proj_hidden': 32,
        'activation': 'prelu',
        'base_model': 'GCNConv',
        'num_layers': 2,
        'drop_edge_rate_1': 0.3,
        'drop_edge_rate_2': 0.4,
        'drop_feature_rate_1': 0.1,
        'drop_feature_rate_2': 0.0,
        'tau': 0.4,
        'num_epochs': 3000,
        'weight_decay': 1e-5,
        'drop_scheme': 'degree',
    }
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay']
    )
    t1 = time.time()
    for epoch in range(1, param['num_epochs'] + 1):
        model.train()
        optimizer.zero_grad()

        edge_index_1 = dropout_adj(data.edge_index, p=param[f'drop_edge_rate_{1}'])[0]
        edge_index_2 = dropout_adj(data.edge_index, p=param[f'drop_edge_rate_{2}'])[0]
        x_1 = drop_feature(data.x, param['drop_feature_rate_1'])
        x_2 = drop_feature(data.x, param['drop_feature_rate_2'])

        z1 = model(x_1, edge_index_1)
        z2 = model(x_2, edge_index_2)

        loss = model.loss(z1, z2)
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print(f'(T) | Epoch={epoch:03d}, loss={loss:.4f}')
    print(f"pretrain time {time.time()-t1:.2f} s, loss {loss:.4f}", flush=True)

In [22]:
for dataset in  ["Cora", "Citeseer", "Pubmed"]:
    print(f'################### {dataset} #################')
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        evaluator = Evaluator(name=f'ogbl-ppa')
    else:
        evaluator = Evaluator(name=f'ogbl-{args.dataset}')
    for r in range(hp['runs']):
        set_seed(r)
        run_grace(r, dataset, evaluator, hp)

################### Cora #################
2708 tensor(2707)
dataset split 
train edge 3696
valid edge 527
valid edge_neg 1055
test edge 1055
test edge_neg 1055


/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)
/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


(T) | Epoch=010, loss=7.1178
(T) | Epoch=020, loss=6.9902
(T) | Epoch=030, loss=6.9279
(T) | Epoch=040, loss=6.8804
(T) | Epoch=050, loss=6.8625
(T) | Epoch=060, loss=6.8371
(T) | Epoch=070, loss=6.8243
(T) | Epoch=080, loss=6.8099
(T) | Epoch=090, loss=6.8018
(T) | Epoch=100, loss=6.8029
(T) | Epoch=110, loss=6.7826
(T) | Epoch=120, loss=6.7881
(T) | Epoch=130, loss=6.7815
(T) | Epoch=140, loss=6.7715
(T) | Epoch=150, loss=6.7691
(T) | Epoch=160, loss=6.7640
(T) | Epoch=170, loss=6.7614
(T) | Epoch=180, loss=6.7596
(T) | Epoch=190, loss=6.7603
(T) | Epoch=200, loss=6.7551
(T) | Epoch=210, loss=6.7554
(T) | Epoch=220, loss=6.7496
(T) | Epoch=230, loss=6.7556
(T) | Epoch=240, loss=6.7525
(T) | Epoch=250, loss=6.7502
(T) | Epoch=260, loss=6.7436
(T) | Epoch=270, loss=6.7434
(T) | Epoch=280, loss=6.7474
(T) | Epoch=290, loss=6.7436
(T) | Epoch=300, loss=6.7416
(T) | Epoch=310, loss=6.7439
(T) | Epoch=320, loss=6.7392
(T) | Epoch=330, loss=6.7390
(T) | Epoch=340, loss=6.7431
(T) | Epoch=35

/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


3327 tensor(3325)
dataset split 
train edge 3187
valid edge 455
valid edge_neg 910
test edge 910
test edge_neg 910


...

(T) | Epoch=010, loss=7.2078
(T) | Epoch=020, loss=7.0923
(T) | Epoch=030, loss=7.0279
(T) | Epoch=040, loss=6.9894
(T) | Epoch=050, loss=6.9737
(T) | Epoch=060, loss=6.9624
(T) | Epoch=070, loss=6.9552
(T) | Epoch=080, loss=6.9501
(T) | Epoch=090, loss=6.9438
(T) | Epoch=100, loss=6.9354
(T) | Epoch=110, loss=6.9342
(T) | Epoch=120, loss=6.9298
(T) | Epoch=130, loss=6.9235
(T) | Epoch=140, loss=6.9263
(T) | Epoch=150, loss=6.9200
(T) | Epoch=160, loss=6.9199
(T) | Epoch=170, loss=6.9189
(T) | Epoch=180, loss=6.9172
(T) | Epoch=190, loss=6.9175
(T) | Epoch=200, loss=6.9141
(T) | Epoch=210, loss=6.9128
(T) | Epoch=220, loss=6.9173
(T) | Epoch=230, loss=6.9157
(T) | Epoch=240, loss=6.9164
(T) | Epoch=250, loss=6.9150
(T) | Epoch=260, loss=6.9137
(T) | Epoch=270, loss=6.9120
(T) | Epoch=280, loss=6.9117
(T) | Epoch=290, loss=6.9131
(T) | Epoch=300, loss=6.9109
(T) | Epoch=310, loss=6.9114
(T) | Epoch=320, loss=6.9091
(T) | Epoch=330, loss=6.9116
(T) | Epoch=340, loss=6.9127
(T) | Epoch=35

/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


19717 tensor(19716)
dataset split 
train edge 31028
valid edge 4432
valid edge_neg 8864
test edge 8864
test edge_neg 8864


/opt/conda/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


(T) | Epoch=010, loss=9.3304
(T) | Epoch=020, loss=9.2434
(T) | Epoch=030, loss=9.1616
(T) | Epoch=040, loss=9.0995
(T) | Epoch=050, loss=9.0335
(T) | Epoch=060, loss=9.0172
(T) | Epoch=070, loss=8.9989
(T) | Epoch=080, loss=8.9646
(T) | Epoch=090, loss=8.9748
(T) | Epoch=100, loss=8.9443
(T) | Epoch=110, loss=8.9510
(T) | Epoch=120, loss=8.9238
(T) | Epoch=130, loss=8.9231
(T) | Epoch=140, loss=8.9301
(T) | Epoch=150, loss=8.9047
(T) | Epoch=160, loss=8.9105
(T) | Epoch=170, loss=8.9096
(T) | Epoch=180, loss=8.9213
(T) | Epoch=190, loss=8.9258
(T) | Epoch=200, loss=8.8993
(T) | Epoch=210, loss=8.9056
(T) | Epoch=220, loss=8.8907
(T) | Epoch=230, loss=8.9062
(T) | Epoch=240, loss=8.9030
(T) | Epoch=250, loss=8.8965
(T) | Epoch=260, loss=8.8980
(T) | Epoch=270, loss=8.8973
(T) | Epoch=280, loss=8.8962
(T) | Epoch=290, loss=8.9085
(T) | Epoch=300, loss=8.8830
(T) | Epoch=310, loss=8.8908
(T) | Epoch=320, loss=8.8952
(T) | Epoch=330, loss=8.8824
(T) | Epoch=340, loss=8.8821
(T) | Epoch=35

## GRACE vs GCN

**Cora**
1. GCN
```python Hits@20
Run: 01, Epoch: 100, Train: 99.81%, Valid: 72.30%, Test: 66.35%
Hits@50
Run: 01, Epoch: 100, Train: 99.95%, Valid: 84.06%, Test: 80.85%
Hits@100
Run: 01, Epoch: 100, Train: 100.00%, Valid: 91.08%, Test: 88.25%
```
2. GRACE
```python Hits@20
Run: 01, Epoch: 100, Train: 100.00%, Valid: 64.71%, Test: 60.19%
Hits@50
Run: 01, Epoch: 100, Train: 100.00%, Valid: 74.19%, Test: 73.65%
Hits@100
Run: 01, Epoch: 100, Train: 100.00%, Valid: 80.08%, Test: 80.76%
```

**Citeseer**
1. GCN
```python Hits@20
Run: 01, Epoch: 100, Train: 100.00%, Valid: 79.12%, Test: 77.58%
Hits@50
Run: 01, Epoch: 100, Train: 100.00%, Valid: 85.71%, Test: 85.49%
Hits@100
Run: 01, Epoch: 100, Train: 100.00%, Valid: 92.09%, Test: 91.10%
  ```

2. GRACE
```python Hits@20
Run: 01, Epoch: 100, Train: 100.00%, Valid: 67.47%, Test: 67.58%
Hits@50
Run: 01, Epoch: 100, Train: 100.00%, Valid: 75.82%, Test: 79.67%
Hits@100
Run: 01, Epoch: 100, Train: 100.00%, Valid: 81.98%, Test: 84.40%
  ```

**PubMed**
1. GCN
```python Hits@20
Run: 01, Epoch: 100, Train: 47.86%, Valid: 43.91%, Test: 36.92%
Hits@50
Run: 01, Epoch: 100, Train: 70.48%, Valid: 59.84%, Test: 59.06%
Hits@100
Run: 01, Epoch: 100, Train: 85.89%, Valid: 72.50%, Test: 75.55%
  ```

3. GRACE
```python Hits@20
Run: 01, Epoch: 100, Train: 91.67%, Valid: 49.62%, Test: 57.68%
Hits@50
Run: 01, Epoch: 100, Train: 99.31%, Valid: 66.49%, Test: 68.87%
Hits@100
Run: 01, Epoch: 100, Train: 99.96%, Valid: 76.17%, Test: 77.00%
```

=> As we can see, GRACE encoder is worst than GCN on Cora and Citeseer, but better on PubMeb, a dense graph.

## GCA Encoder

In [65]:
def run_gca(r, dataset, evaluator, hp, drop_scheme='degree'):
    writer = SummaryWriter(f"./rec/GCA_NCN")
    writer.add_text("hyperparams", str(hp)) 
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        data, split_edge = loaddataset(dataset, False) # get a new split of dataset
        data = data.to(device)
    bestscore = None
    # build model
    basic_encoder = Encoder(data.num_features, hp['hiddim'], nn.Identity()).to(device)
    model = GRACE(basic_encoder, hp['hiddim'], 32).to(device)
    pretrain_gca(model, data, drop_scheme)
    predictor = CNLinkPredictor(hp['hiddim'], hp['hiddim'], 1, 3,
                       hp['predp'], hp['preedp'], hp['lnnn']).to(device)
    optimizer = torch.optim.Adam([{'params': model.parameters(), "lr": hp['gnnlr']}, 
       {'params': predictor.parameters(), 'lr': hp['prelr']}])

    for epoch in range(1, 1 + hp['epochs']):
        legacy_train(epoch, model, predictor, data, split_edge, optimizer, evaluator, hp)
        if epoch % 100 == 0:
            legacy_test(r, epoch, model, predictor, data, split_edge, evaluator, bestscore, writer, hp)

In [66]:
def pretrain_gca(model, data, drop_scheme='degree'):
    param = {
        'learning_rate': 0.01,
        'num_hidden': 256,
        'num_proj_hidden': 32,
        'activation': 'prelu',
        'base_model': 'GCNConv',
        'num_layers': 2,
        'drop_edge_rate_1': 0.3,
        'drop_edge_rate_2': 0.4,
        'drop_feature_rate_1': 0.1,
        'drop_feature_rate_2': 0.0,
        'tau': 0.4,
        'num_epochs': 3000,
        'weight_decay': 1e-5,
        'drop_scheme': drop_scheme,
    }
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=param['learning_rate'],
        weight_decay=param['weight_decay']
    )

    # compute drop_weights per centrality metrics
    if param['drop_scheme'] == 'degree':
        drop_weights = degree_drop_weights(data.edge_index).to(device)
    elif param['drop_scheme'] == 'pr':
        drop_weights = pr_drop_weights(data.edge_index, aggr='sink', k=200).to(device)
    elif param['drop_scheme'] == 'evc':
        drop_weights = evc_drop_weights(data).to(device)
    else:
        drop_weights = None

    # # compute feature_weights per centrality metrics
    if param['drop_scheme'] == 'degree':
        edge_index_ = to_undirected(data.edge_index)
        node_deg = degree(edge_index_[1])
        feature_weights = feature_drop_weights(data.x, node_c=node_deg).to(device)
    elif param['drop_scheme'] == 'pr':
        node_pr = compute_pr(data.edge_index)
        feature_weights = feature_drop_weights(data.x, node_c=node_pr).to(device)
    elif param['drop_scheme'] == 'evc':
        node_evc = eigenvector_centrality(data)
        feature_weights = feature_drop_weights(data.x, node_c=node_evc).to(device)
    else:
        feature_weights = torch.ones((data.x.size(1),)).to(device)
    
    t1 = time.time()
    for epoch in range(1, param['num_epochs'] + 1):
        model.train()
        optimizer.zero_grad()
        edge_index_1 = drop_edge_weighted(data.edge_index, drop_weights, p=param[f'drop_edge_rate_{1}'], threshold=0.7)
        edge_index_2 = drop_edge_weighted(data.edge_index, drop_weights, p=param[f'drop_edge_rate_{2}'], threshold=0.7)
        x_1 = drop_feature_weighted_2(data.x, feature_weights, param['drop_feature_rate_1'])
        x_2 = drop_feature_weighted_2(data.x, feature_weights, param['drop_feature_rate_2'])

        z1 = model(x_1, edge_index_1)
        z2 = model(x_2, edge_index_2)

        loss = model.loss(z1, z2)
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print(f'(T) | Epoch={epoch:03d}, loss={loss:.4f}')
    print(f"pretrain time {time.time()-t1:.2f} s, loss {loss:.4f}", flush=True)

In [67]:
for dataset in  ["Citeseer", "Cora", "Pubmed"]:
    print(f'################### {dataset} #################')
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        evaluator = Evaluator(name=f'ogbl-ppa')
    else:
        evaluator = Evaluator(name=f'ogbl-{args.dataset}')
    for r in range(hp['runs']):
        set_seed(r)
        run_gca(r, dataset, evaluator, hp, 'degree')

################### Citeseer #################
3327 tensor(3325)
dataset split 
train edge 3187
valid edge 455
valid edge_neg 910
test edge 910
test edge_neg 910


/home/jovyan/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


RuntimeError: size mismatch, got 3703, 3703x3327,3326

In [68]:
for dataset in  ["Cora", "Citeseer", "Pubmed"]:
    print(f'################### {dataset} #################')
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        evaluator = Evaluator(name=f'ogbl-ppa')
    else:
        evaluator = Evaluator(name=f'ogbl-{args.dataset}')
    for r in range(hp['runs']):
        set_seed(r)
        run_gca(r, dataset, evaluator, hp, 'pr')

################### Cora #################
2708 tensor(2707)
dataset split 
train edge 3696
valid edge 527
valid edge_neg 1055
test edge 1055
test edge_neg 1055


/home/jovyan/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


(T) | Epoch=100, loss=7.4660
(T) | Epoch=200, loss=7.4343
(T) | Epoch=300, loss=7.4506
(T) | Epoch=400, loss=7.3313
(T) | Epoch=500, loss=7.3663
(T) | Epoch=600, loss=7.3358
(T) | Epoch=700, loss=7.3502
(T) | Epoch=800, loss=7.2739
(T) | Epoch=900, loss=7.3093
(T) | Epoch=1000, loss=7.2697
(T) | Epoch=1100, loss=7.3093
(T) | Epoch=1200, loss=7.2348
(T) | Epoch=1300, loss=7.2604
(T) | Epoch=1400, loss=7.3037
(T) | Epoch=1500, loss=7.2455
(T) | Epoch=1600, loss=7.2400
(T) | Epoch=1700, loss=7.2233
(T) | Epoch=1800, loss=7.2111
(T) | Epoch=1900, loss=7.2442
(T) | Epoch=2000, loss=7.2504
(T) | Epoch=2100, loss=7.1836
(T) | Epoch=2200, loss=7.1765
(T) | Epoch=2300, loss=7.1965
(T) | Epoch=2400, loss=7.2084
(T) | Epoch=2500, loss=7.1692
(T) | Epoch=2600, loss=7.1969
(T) | Epoch=2700, loss=7.1868
(T) | Epoch=2800, loss=7.1643
(T) | Epoch=2900, loss=7.1724
(T) | Epoch=3000, loss=7.1834
pretrain time 45.39 s, loss 7.1834
10 train time 0.06 s, loss 0.4126
10 train time 0.06 s, loss 0.2851
10 tra

/home/jovyan/envs/ncn/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


3327 tensor(3325)
dataset split 
train edge 3187
valid edge 455
valid edge_neg 910
test edge 910
test edge_neg 910


RuntimeError: size mismatch, got 3703, 3703x3327,3326

In [ ]:
for dataset in  ["Cora", "Citeseer", "Pubmed"]:
    print(f'################### {dataset} #################')
    if dataset in ["Cora", "Citeseer", "Pubmed"]:
        evaluator = Evaluator(name=f'ogbl-ppa')
    else:
        evaluator = Evaluator(name=f'ogbl-{args.dataset}')
    for r in range(hp['runs']):
        set_seed(r)
        run_gca(r, dataset, evaluator, hp, 'pr')

## BGRL encoder